# a2 - python - CS4821

This assignment will cover topics from data, data preprocessing, and classification.

Make sure that you keep this notebook named as "a2-4821.ipynb" 

Submit the zip-file created after running your notebook on the Linux lab machines.

Any other packages or tools, outside those listed in the assignments or Canvas, should be cleared
by Dr. Brown before use in your submission.

## Q0 - Setup

The following code looks to see whether your notebook is run on Gradescope (GS), Colab (COLAB), or the linux Python environment you were asked to setup.

In [2]:
import re 
import os
import platform 
import sys 

# flag if notebook is running on Gradescope 
if re.search(r'amzn', platform.uname().release): 
    GS = True
else: 
    GS = False

# flag if notebook is running on Colaboratory 
try:
  import google.colab
  COLAB = True
except:
  COLAB = False

# flag if running on Linux lab machines. 
cname = platform.uname().node
if re.search(r'(guardian|colossus|c28|coc-15954-m)', cname):
    LLM = True 
else: 
    LLM = False

print("System: GS - %s, COLAB - %s, LLM - %s" % (GS, COLAB, LLM))

System: GS - False, COLAB - False, LLM - True


In [3]:
# Import standard DS packages 
import pandas as pd
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import math
import statistics
import textwrap
%matplotlib inline

from sklearn import tree        # decision tree classifier
from sklearn import neighbors   # knn classifier
from sklearn import naive_bayes # naive bayes classifier 
from sklearn import metrics     # performance evaluation metrics
from sklearn import model_selection
from sklearn import preprocessing 
from sklearn import pipeline
# import graphviz, pydotplus

from sklearn.model_selection import train_test_split, StratifiedKFold 
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn import tree
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import f1_score, roc_auc_score,mean_squared_error
from sklearn.metrics import confusion_matrix

# Package for Autograder 

import os
if os.environ["HOME"]=='/home/jovyan':
    !pip install --upgrade otter-grader
    
import otter 
grader = otter.Notebook()

from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

# Q1 - Exploratory Data Analysis 

Consider the `movies` data set available on Canvas. The data set is made up of over 600 randomly selected movies, released before 2016, with information extracted from IMDB and Rotten Tomatoes. A code book on the variables is also provided.

You should explore the files a bit in a text editor to understand the format. The variables are made up of different types: nominal, ordinal, numeric, etc. We will refer to the different variables by their column / codebook names.


<!-- BEGIN QUESTION -->

# *Missing Data*

## Q1(a) - Examine data for loading 

Look at the `movies` data set.  Is there any missing data in the `movies` data set? 
If so, how is it encoded? 

In [16]:
movies = pd.read_csv("movies.csv")
sum(movies.isnull().values) # find the indexes with missing values

,title,title_type,genre,runtime,mpaa_rating,studio,thtr_rel_year,thtr_rel_month,thtr_rel_day,dvd_rel_year,...,best_dir_win,top200_box,director,actor1,actor2,actor3,actor4,actor5,imdb_url,rt_url
0,Filly Brown,Feature Film,Drama,80.0,R,Indomina Media Inc.,2013,4,19,2013.0,...,no,no,Michael D. Olmos,Gina Rodriguez,Jenni Rivera,Lou Diamond Phillips,Emilio Rivera,Joseph Julian Soria,http://www.imdb.com/title/tt1869425/,//www.rottentomatoes.com/m/filly_brown_2012/
1,The Dish,Feature Film,Drama,101.0,PG-13,Warner Bros. Pictures,2001,3,14,2001.0,...,no,no,Rob Sitch,Sam Neill,Kevin Harrington,Patrick Warburton,Tom Long,Genevieve Mooy,http://www.imdb.com/title/tt0205873/,//www.rottentomatoes.com/m/dish/
2,Waiting for Guffman,Feature Film,Comedy,84.0,R,Sony Pictures Classics,1996,8,21,2001.0,...,no,no,Christopher Guest,Christopher Guest,Catherine O'Hara,Parker Posey,Eugene Levy,Bob Balaban,http://www.imdb.com/title/tt0118111/,//www.rottentomatoes.com/m/waiting_for_guffman/
3,The Age of Innocence,Feature Film,Drama,139.0,PG,Columbia Pictures,1993,10,1,2001.0,...,yes,no,Martin Scorsese,Daniel Day-Lewis,Michelle Pfeiffer,Winona Ryder,Richard E. Grant,Alec McCowen,http://www.imdb.com/title/tt0106226/,//www.rottentomatoes.com/m/age_of_innocence/
4,Malevolence,Feature Film,Horror,90.0,R,Anchor Bay Entertainment,2004,9,10,2005.0,...,no,no,Stevan Mena,Samantha Dark,R. Brandon Johnson,Brandon Johnson,Heather Magee,Richard Glover,http://www.imdb.com/title/tt0388230/,//www.rottentomatoes.com/m/10004684-malevolence/


In [30]:
# investigate missing values one feature at a time
idx3 = movies[movies.iloc[:,3].isnull() == True]
idx3


,title,title_type,genre,runtime,mpaa_rating,studio,thtr_rel_year,thtr_rel_month,thtr_rel_day,dvd_rel_year,...,best_dir_win,top200_box,director,actor1,actor2,actor3,actor4,actor5,imdb_url,rt_url
328,The End of America,Documentary,Documentary,NaN,Unrated,Indipix,2008,10,1,2009.0,...,no,no,Anne Sundberg,Naomi Wolf,NaN,NaN,NaN,NaN,http://www.imdb.com/title/tt1294790/,//www.rottentomatoes.com/m/end_of_america/


In [33]:
idx5 = movies[movies.iloc[:,5].isnull() == True]
idx5
# the trend becomes clear, missing values are recorded as np.NaN

,title,title_type,genre,runtime,mpaa_rating,studio,thtr_rel_year,thtr_rel_month,thtr_rel_day,dvd_rel_year,...,best_dir_win,top200_box,director,actor1,actor2,actor3,actor4,actor5,imdb_url,rt_url
92,Dirty Sanchez: The Movie,Documentary,Comedy,94.0,R,NaN,2007,9,11,2007.0,...,no,no,Jim Hickey,Lee Dainton,Dan Joyce,Michael Pancho Locke,Matt Pritchard,Howard Marks,http://www.imdb.com/title/tt0758743/,//www.rottentomatoes.com/m/dirty_sanchez/
204,Caveman,Feature Film,Action & Adventure,91.0,PG,NaN,1981,4,17,2002.0,...,no,no,Carl Gottlieb,Ringo Starr,Dennis Quaid,Shelley Long,Jack Gilford,John Matuszak,http://www.imdb.com/title/tt0082146/,//www.rottentomatoes.com/m/caveman/
277,Oliver & Company,Feature Film,Animation,74.0,G,NaN,1988,11,18,2002.0,...,no,no,George Scribner,Joey Lawrence,Billy Joel,Cheech Marin,Bette Midler,Dom DeLuise,http://www.imdb.com/title/tt0095776/,//www.rottentomatoes.com/m/oliver_and_company/
302,The Man Who Sued God,Feature Film,Drama,97.0,R,NaN,2001,10,25,2007.0,...,no,no,Mark Joffe,Billy Connolly,Judy Davis,Colin Friels,Bille Brown,Wendy Hughes,http://www.imdb.com/title/tt0268437/,//www.rottentomatoes.com/m/man_who_sued_god/
396,Inserts,Feature Film,Drama,117.0,NC-17,NaN,1975,1,1,2005.0,...,no,no,John Byrum,Richard Dreyfuss,Jessica Harper,Stephen Davies,Veronica Cartwright,Bob Hoskins,http://www.imdb.com/title/tt0073172/,//www.rottentomatoes.com/m/inserts/
461,Inbred,Feature Film,Art House & International,90.0,R,NaN,2011,8,29,2013.0,...,no,no,Alex Chandon,James Burrows,James Doherty,Jo Hartley,Seamus O'Neill,Neil Leiper,http://www.imdb.com/title/tt1723124/,//www.rottentomatoes.com/m/inbred/
530,Death Line (Raw Meat),Feature Film,Horror,87.0,R,NaN,1972,1,1,2003.0,...,no,no,Gary Sherman,Hugh Armstrong,James Cossins,Sharon Gurney,David Ladd,Christopher Lee,http://www.imdb.com/title/tt0068458/,//www.rottentomatoes.com/m/death_line/


Missing data is encoded as: ***np.NaN values***

<!-- END QUESTION -->

## Q1(b) - Load the data

Load the movies data into a DataFrame `q1movies`.  Is there any missing data in the `movies` data set? 
If yes, make sure to encode those missing values when reading the data in pandas `read_csv` function. 


In [38]:
# Read in movies data with pandas "read_csv" function
#  Use column names from the original csv file 

q1movies = pd.read_csv("movies.csv", na_values='MISSING')

q1movies.head()

,title,title_type,genre,runtime,mpaa_rating,studio,thtr_rel_year,thtr_rel_month,thtr_rel_day,dvd_rel_year,...,best_dir_win,top200_box,director,actor1,actor2,actor3,actor4,actor5,imdb_url,rt_url
0,Filly Brown,Feature Film,Drama,80.0,R,Indomina Media Inc.,2013,4,19,2013.0,...,no,no,Michael D. Olmos,Gina Rodriguez,Jenni Rivera,Lou Diamond Phillips,Emilio Rivera,Joseph Julian Soria,http://www.imdb.com/title/tt1869425/,//www.rottentomatoes.com/m/filly_brown_2012/
1,The Dish,Feature Film,Drama,101.0,PG-13,Warner Bros. Pictures,2001,3,14,2001.0,...,no,no,Rob Sitch,Sam Neill,Kevin Harrington,Patrick Warburton,Tom Long,Genevieve Mooy,http://www.imdb.com/title/tt0205873/,//www.rottentomatoes.com/m/dish/
2,Waiting for Guffman,Feature Film,Comedy,84.0,R,Sony Pictures Classics,1996,8,21,2001.0,...,no,no,Christopher Guest,Christopher Guest,Catherine O'Hara,Parker Posey,Eugene Levy,Bob Balaban,http://www.imdb.com/title/tt0118111/,//www.rottentomatoes.com/m/waiting_for_guffman/
3,The Age of Innocence,Feature Film,Drama,139.0,PG,Columbia Pictures,1993,10,1,2001.0,...,yes,no,Martin Scorsese,Daniel Day-Lewis,Michelle Pfeiffer,Winona Ryder,Richard E. Grant,Alec McCowen,http://www.imdb.com/title/tt0106226/,//www.rottentomatoes.com/m/age_of_innocence/
4,Malevolence,Feature Film,Horror,90.0,R,Anchor Bay Entertainment,2004,9,10,2005.0,...,no,no,Stevan Mena,Samantha Dark,R. Brandon Johnson,Brandon Johnson,Heather Magee,Richard Glover,http://www.imdb.com/title/tt0388230/,//www.rottentomatoes.com/m/10004684-malevolence/


In [37]:
grader.check("q1b")

q1b results: All test cases passed!

## Q1(c) - Missing data 

We want to understand where (which variable) and how much data is missing (for each variable the percentage of rows). 

In [ ]:
# Create a Data.Series that has the percentage of missing data for each 
#  attribute in the movies data set.
miss_data = ... 

miss_data

In [ ]:
grader.check("q1c")

## Q1(d) - Clean data 

We want to clean up data with respect to the missing values. 

Ignore any missing values in the `studio`, `dvd_rel_year`, `dvd_rel_month`, `dvd_rel_day`, and all variables including and listed after `best_pic_nom`. For other missing values, remove the sample that contains the missing value.

Save the resulting DataFrame in the `movies` variable.  

In [ ]:

# Ignore missing values in "studio", "dvd_rel_year", "dvd_rel_month", 
#   "dvd_rel_day", and all varaibles including and after "best_pic_nom"
# For other missing values, remove the sample that contains the missing value. 

movies = ...

movies.shape

In [ ]:
grader.check("q1d")

## Q1(e) - Attribute Types

For the following variables, state the attribute type: 1- *nominal*, 2- *ordinal*, 3- *interval*, or 4- *ratio*. 

In [ ]:
type_genre = ...
type_runtime = ...
type_mpaa_rating = ...
type_studio = ...
type_thtr_rel_month = ...
type_imdb_rating = ...
type_audience_score = ...
type_best_pic_win = ...

In [ ]:
grader.check("q1e")

# *Summary Statistics*

## Q1(f) - Statistics, part 1

For the following variables, report out a five number summary:
`audience_score` and `imdb_rating`

Store results in a DataFrame: `q1f`

*Hint:* consider using the `describe` function.


In [ ]:

# Report five number summary for variables `audience_score` and `imdb_rating` in 
#  a DataFrame "q1f" 
#  Rows should represent: min, Q1 - 25%, Q2 - 50%, Q3 - 50%, max 
#  Columns should be `audience_score` then `imdb_rating` 

q1f = ...

q1f

In [ ]:
grader.check("q1f")

## Q1(g) - Statistics, part 2 

Report the mean, median, and mode of `critics_score` and `runtime` to the given variables. 

In [ ]:
# Report mean, median and mode of "critics_score" and "runtime" 

# For critics_score- 
q1g_cs_mean = ...
q1g_cs_median = ...
q1g_cs_mode = ...

# For runtime- 
q1g_r_mean = ...
q1g_r_median = ...
q1g_r_mode = ...


In [ ]:
grader.check("q1g")

## Q1(h) - Statistics, part 3

Report the first quartile, 37th percentile, third quartile, and 83rd percentile
for `critics_score` and `runtime` and assign it to the given variables.  

In [ ]:
# Report first quartile, 31st percentile, third quartile, and 90th percentile
#  of "critics_score" and "runtime" 

# For critics_score- 
q1h_cs_q1 = ...
q1h_cs_p37 = ...
q1h_cs_q3 = ...
q1h_cs_p83 = ...

# For runtime- 
q1h_r_q1 = ...
q1h_r_p37 = ...
q1h_r_q3 = ...
q1h_r_p83 = ...


In [ ]:
grader.check("q1h")

<!-- BEGIN QUESTION -->

# *Visualizations* 

## Q1(i) Visualization: Single Variable 

I highly encourage looking at the [Fundamentals of Visualization](https://clauswilke.com/dataviz/index.html) reference book to guide in the creation of “good” visualizations requested below.


Create a bar plot for the `critics_rating` variable.

In [ ]:

# Create bar plot for "critics_rating" 




<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Q1(j) Visualization: Two Variables 

Create a violin plot for `imdb_rating` grouped by `mpaa_rating` (sorted by mpaa rating, where 'Unrated' is last).



In [ ]:

# Create a violin plot for `imdb_rating` grouped by `mpaa_rating` (sorted)




<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Q1(k) Visualization: Multiple variables

Create a stacked bar chart to display the proportion of wins (nominations) for the 5 `best_*` variables.


In [ ]:

# Create a stacked bar chart to display the proportion of wins (nominations) 
#   for the 5 `best_*` variables.




<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Q1(l) Visualization: Two Variables 

Create an overlapping density plot for `critics_score` grouped by `audience_rating`. 

In [ ]:
# Create overlapping density plot




<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Q1(m) Visualization: Multiple variables 

Create a small multiples (or faceted) scatter plot of `imdb_rating` (y-axis) against `runtime` (x-axis) for each `critics_rating`.


In [ ]:

# Create a small multiples (or faceted) scatter plot of `imdb_rating` vs. 
#   `runtime` for each `critics_rating`.




<!-- END QUESTION -->

<!-- BEGIN QUESTION -->

## Q1(bonus)   

Create a small multiples (or faceted) scatter plot of `imdb_rating` vs. `runtime` for each of the top 4 `genre` (in order of most frequent), colored with the `critics_rating`.

In [ ]:

# Create a small multiples (or faceted) scatter plot of `imdb_rating` vs. 
#   `runtime` for each of the top 4 `genre`, colored by the `critics_rating`





<!-- END QUESTION -->

# Q2 - Data Preprocessing

Let's make sure you become familiar using the scaling functions that are available in standard libraries: 

* Python: `MinMaxScaler` and `StandardScaler` or scale in `sklearn.preprocessing`

Here you will use the `MinMaxScaler`.


In [ ]:
# train/test data 
q2train = pd.DataFrame({'x1': [20, 37, 40, 60, 85, 120], 
                        'x2': [-10, -8, 52, 3, 18, 23]})
q2test = pd.DataFrame({'x1': [42, 58, 101], 'x2': [-8, 42, 54]})

# Setup a scaler, fit and transform the training data 
scaler = preprocessing.MinMaxScaler()
q2outA = scaler.fit_transform(q2train)
print(q2outA)


# Setup a MinMaxScaler with default range parameters
# fit the scaler to the training data, transform it, and transform the test data
scaler = ...
q2outBtrain = ...
q2outBtest = ...
print(q2outBtrain)
print(q2outBtest)



In [ ]:
grader.check("q2")

# Q3 - Data Preprocessing

Let's make sure you become familiar using the scaling functions that are available in standard libraries: 

* Python: `MinMaxScaler` and `StandardScaler` or scale in `sklearn.preprocessing`

Here you will use the `StandardScaler`.

In [ ]:
# train/test data 
# q2train = pd.DataFrame({'x1': [20, 37, 40, 60, 85, 120], 
#                       'x2': [-10, -8, 52, 3, 18, 23]})
# q2test = pd.DataFrame({'x1': [42, 58, 101], 'x2': [-8, 42, 54]})


# Setup a Standard scaler, fit and transform the training data 
scaler = preprocessing.StandardScaler()
q3outA = scaler.fit_transform(q2train)
print(q3outA)

# use StandardScaler
# fit the scaler to the training data, transform it, and transform the test data
scaler = ...
q3outBtrain = ...
q3outBtest = ...
print(q3outBtrain)
print(q3outBtest)


In [ ]:
grader.check("q3")

# Q4 - Performance Metrics 

Write a function to calculate: 
(a) true positive rate,
(b) false positive rate, 
(c) accuracy, and 
(d) Matthews Correlation Coefficient (MCC). 

You can make use of `sklearn.metrics` functions. 

The function will have inputs of `y_true` (np.array) - the true label for a set of samples and `y_pred` (np.array) - the predicted labels for a set of samples, and a threshold `thres_value` (float). 

The function returns a list of the true positive rate, false positive rate, accuracy and MCC for the inputs where the predicted labels are thresholded at the provided value (using >= comparisons). 


This function will then be used to create a DataFrames  with rows corresponding with the 10 thresholds (y_pred values) and columns reporting the different thresholds, the true positive rate (TPR), false positive rate (FPR), accuracy (ACC), and Matthews correlation coefficient (MCC).



In [ ]:

def calc_metrics(y_true, y_pred, thres_value):
    # Calculate tpr, fpr, accuracy, and MCC on input
    # Input: 
    #  y_true - sample labels       (np.array)
    #  y_pred - sample predictions  (np.array)
    #  thres_value - threshold for predictions,  >= 
    # Return list of tpr, fpr, accuracy, and MCC 
    
    ...
    return ...



y_true = np.array([1,1,0,1,1,0,1,0,0,0])
y_pred = np.array([0.98,0.92,0.85,0.77,0.71,0.64,0.57,0.42,0.34,0.32])


perfDF = pd.DataFrame(columns = ['Threshold', 'TPR', 'FPR', 'ACC', 'MCC']) 
i = 0 
for thres in y_pred: 
    tpr_val, fpr_val, acc_val, mcc_val = calc_metrics(y_true, y_pred, thres)
    perfDF.loc[i] = [thres, tpr_val, fpr_val, acc_val, mcc_val]
    i = i+1

perfDF

In [ ]:
grader.check("q4")

<!-- BEGIN QUESTION -->

# Q5 - Plot ROC Curve:

Use the results from Question 4 to plot the ROC curve for the data. 

Note, plot this curve using the standard plotting tools rather than any special library/package available for making ROC plots.

In [ ]:

# Create a ROC curve using the results from Q4





<!-- END QUESTION -->

# Q6 - NBA Rookies

For this problem you will use a data set of rookie NBA players from 1980 - 2017 seasons.  The dataset was collected from the NBA website API - https://www.nba.com. 

You will use the data from their rookie year to predict whether a player will last at least 5 seasons in the leaque. 

The data consists of variables:

* `PlayerID`, `Player` - variables to identify individual samples (ignore for prediction)
* `Tm`, `Year` - variables describing the year the player started and for what team (ignore for prediction)  
* `TARGET` - This is the target / class feature to be predicted (whether the player was in the league for at least 5 years). 

The remaining variables are predictor variables for the models.  They come in pairs "*\_DIFF" and "*\_A" reporting the given statistic as the difference between Team A and Team B and the statistic itself for Team A. 

* `Pos` - position of the player, power forward, point guard, shooting guard, center, etc.
* `Age` - player age
* `G` - sum of number of games played
* `GS` - sum of number of games started
* `MP` - sum of number of minutes played
* `PTS` - sum of number of points scored
* `FG` - sum of number of field goals made (both 2 and 3 pointers)
* `FGA` - sum of number of field goals attempted
* `FG%` - FG / FGA, percentage of field goals made 
* `3P` - sum of the number of 3 pointers made
* `3PA` - sum of the number of 3 pointers attempted
* `3P%` - 3P / 3PA, percentage of 3 pointers made
* `2P` - sum of the number of 2 point shots made
* `2PA` - sum of the number of 2 point shots attempted
* `2P%` - 2P / 2PA, percentage of 2 point shots made
* `eFG%` - Effective Field Goal Percentage, (`FG` + 0.5 * `3P`)/`FGA`
* `FT` - sum of the number of free throws made
* `FTA` - sum of the number of free throws attempted
* `FT%` - FT / FTA, percentage of free throws made
* `ORB` - sum of the number of offensive rebounds
* `DRB` - sum of the number of defensive rebounds
* `TRB` - sum of the number of total rebounds
* `AST` - sum of the number of assists
* `STL` - sum of the number of steals
* `BLK` - sum of the number of blocks
* `TOV` - sum of the number of turnovers
* `PF` - sum of the number of personal fouls

More information on the stats used can be found: https://www.nba.com/stats/help/glossary   
*Note, some of the abbreviations used here are slightly different* 


## Q6(a) - Load Data 

Load the `nba` data.


In [ ]:
nba = pd.read_csv(...) 

nba.head()

In [ ]:
grader.check("q6a")

## Q6(b) - Missing Data 

Let's investigate any missing values in the nba data. 

First, calculate and report the percentage of missing data for that variable (percentage of rows) in a DataSeries, `miss_nba`. 

In [ ]:
miss_nba = ... 

miss_nba

In [ ]:
grader.check("q6b")

Explore where these missing values you in the following code cell. 

**BE SURE TO COMMENT OUT YOUR CODE BEFORE SUBMITTING**

In [ ]:
# explore where the missing values are in your data. 


## Q6(c) - Handle missing data

By investigating where the missing data is, I hope you discovered the following: 

The missing values reside in two main types of columns (and for two different reasons):
* `GS` the game started column has missing values for almost all rookies in the 1980 and 1981 seasons with a few exceptions, e.g., Larry Bird, Clint Richardson, etc.
* Columns that calculate a percentage, `FG%`, `3P%`, `2P%`, `eFG%`, `FT%`
Here the missing values are due to a divide by zero.

Due to the different reasons for the data missing values will be handled differently in each situation. 

First, the missing values for `GS`, because there is not way to impute these values (and there are so few players in the 1980 and 1981 season having this information), the **all** samples from these two seasons should be deleted. 

For the missing values in the "percentage" columns, replace those missing values with 0. 

Call you new DataFrame after performing these operations `nba2`.


*Note* There may be situations where how you handle missing data (in particular, using imputation methods) should be done in the workflow/pipeline after already splitting for training/testing data in order to avoid possible data leakage.  However, the methods we employ here for dealing the the missing data are not using global properties to perform the imputation, therefore can be done at this state of the analysis. 

In [ ]:
nba2 = ... 

nba2.head()

In [ ]:
grader.check("q6c")

## Q6(d) - Labels 

Let's understand the what we should expect as a baseline performance for predicting whether the rookie play remains in the league for at least 5 years. 

(i). What fraction of players have a positive target label (in the league for at least 5 years)? Value should be in between 0 and 1.  
(ii).  What should a constant classifier model predict?   A *constant classifier* always predicts the same value no matter the input.   
(iii).  What is the error rate of the constant classifier? Value should be in between 0 and 1. 

Answer the following questions below.  Note, you should not use any `sklearn` functions, but simply look at properties of the data labels. 


In [ ]:

q6d_i = ...

q6d_ii = ...

q6d_iii = ...


In [ ]:
grader.check("q6d")

## Q6(e) - Prepare the data 

At this point, we need to set up the data in order to be used in the classification models. 

We want to create a DataFrame `nbaX` for the predictor variables and `nbaY` for the target variable. 

The target variable, `nbaY` will just be the `Target` column of the `nba2` data set. 

The predictor variables have more considerations. 

We want to exclude player information such as IDs, `PlayerID` and names `Player`, that are identifying and not predictive.   

You should also exclude these other factors (note, that some of these variables may in fact be predictive but we are going to exclude at this time): 

* `POS` - player position
* `Tm` - team
* `Year` - rookie year

Finally, several of the numeric predictive variables are not only related, but can be directly calculated from one another, e.g., `FG%` = `FG` / `FGA`.  Having variables that are closely related, or in this case redundant may actually hinder the predictive models. 

Therefore, exclude the following variables: `FG`, `3P`, `2P`, `eFG%`, `FT`, `TRB`. 

In [ ]:
nbaY = ...
nbaX = ...

print(nbaX.shape)
print(nbaY.shape)

In [ ]:
grader.check("q6e")

## Q6(f) - Model Selection and Evaluation: Three-fold Split

Split the data into training, validation and test sets with 60, 20, and 20% of the data respectively. Make sure to split the data such that the distribution of class labels is approximately equal across splits - “stratify”.

Set the seed for the random generator in `random_state` to ”4821”

In [ ]:
# Split of the test set
X_trainval, X_test, y_trainval, y_test = ...

# Split trainval into train + val 
X_train, X_val, y_train, y_val = ...


In [ ]:
grader.check("q6f")

## Q6(g) - Scaling 

Scale the predictor data with standard scaling (Gaussian normalizaiton). 

Make sure to use training data set to set scaling parameters and apply those parameters to scaling the training and validation data.

Use the train+val to scale the train+val data, and use those parameters to scale the test data to evaluate the best model. 

Helpful functions: Python - `StandardScaler` from `sklearn.preprocessing`.

In [ ]:
scaler = ...
X_train_sc = ...
X_val_sc = ...

scaler_final = ...
X_trainval_sc = ...
X_test_sc = ...


In [ ]:
grader.check("q6g")

## Q6(h) -  KNN - K Nearest Neighbors
For values of k, [3, 7, 11, 15, 19, 23, 27, 31], find the best k-nearest-neighbor classifier using the three-fold split of data. 

* fit a k nearest neighbors model to the training data for each value of k
* evaluate the classifier on the training and validation set using AUC
* select the best value of k
* create a model on train+validation with the *best* value of k 
* evaluate the classifier on the test data. 
* plot the training and validation performance vs k;  
add a line showing the test performance

In [ ]:

kvals = [3, 7, 11, 15, 19, 23, 27, 31]

knn_auc_val = np.zeros((1,len(kvals)))
knn_auc_tr = np.zeros((1,len(kvals)))


# fit a k nearest neighbors model to the training data
# evaluate the classifier on the training and validation set using auc


# select the best value of k
knn_bestk = ...

# create a best model on train+validation

# evaluate the classifier on the test data.
knn_auc_test = ...

# plot the training and validationing performance vs k.
#   add a line for test performance 




print('Best k:  %d' % (knn_bestk))
print('Test Perf:  %.6f' % (knn_auc_test))

In [ ]:
grader.check("q6h")

## Q6(i) - Decision Trees

For values of max_leaf_nodes nodes [5, 10, 25, 50, 75, 100], fit the Decision Trees classifier to the training data. 

- fit a decision tree model to the training data (use `random_state=4821`) for each of the max_leaf_nodes values
- evaluate the classifier on the training and validation set using AUC
- select the best max_leaf_nodes
- retrain the a model on train+validation with the *best* value of max_leaf_nodes 
- report the auc on the testing data.
- plot the train and validation AUC vs max_leaf_nodes;  
add a line for the test AUC performance 
- print out the best tree.

In [ ]:

nodes = [5, 10, 25, 50, 75, 100, 150]

dt_auc_val = np.zeros((1,len(nodes)))
dt_auc_tr = np.zeros((1,len(nodes)))

# fit a decision tree model to the training data (use random_state=4821)
# evaluate the classifier on the training and validation set using auc


# select the best max_leaf_nodes
dt_bestn = ...

# retrain the best model on train+validation
# report the auc on the testing data.
dt_auc_test = ...

# plot the train and validation auc vs max_leaf_nodes.
#   add a line for test performance 


print('Best max_leaf_nodes:  %d' % (dt_bestn))
print('Test Perf:  %.6f' % (dt_auc_test))

In [ ]:

# print out the tree



In [ ]:
grader.check("q6i")

## Q6(j) -  Naive Bayes 

Train a Gaussian Naive Bayes model on training + validation data and report the training+val and testing data performance (auc). 


In [ ]:
# q4j

nb_auc_trainval = ...
nb_auc_test = ...


In [ ]:
grader.check("q6j")

## Submission

Make sure you have run all cells in your notebook in order before running the cell below, so that all images/graphs appear in the output. The cell below will generate a zip file for you to submit. **Please save before exporting!**

**NOTE** the submission must be run on the campus linux machines.  See the instruction in the Canvas assignment.

In [ ]:
# Save your notebook first, then run this cell to export your submission.
grader.export(pdf=False, run_tests=True)